# Basic Samples : apache_age_py

## Prepare
```
import age
```
## Connect to PostgreSQL(with AGE extention)
```
ag = age.connect(graph="(graph name}", host="{host}", port="{port}", dbname="{dbname}", user="{db username}", password="{password}")

or 

ag = age.connect(host="{host}", port="{port}", dbname="{dbname}", user="{db username}", password="{password}")
# If you created a graphPath already.
ag = age.setGraph("{graph name}")
```

In [1]:
import age
from age.gen.ageParser import *

ag = age.connect(graph="my_graph", host="172.17.0.3", port="5432", dbname="postgres", user="postgres", password="agens")


## Create & Query Vertices

### Create Vertices
```
ag.execCypher("CREATE(...)") # Cypher Create Statement
...
ag.commit() # commit
```

### Query Vertices (with result parsed full Graph object)
```
ag.queryCypher("MATCH (n) RETURN n") # Cypher Query
graph = ag.graph()
for vertex in graph:
    print(vertex.label, vertex.properties["name"], vertex) # row has id, label, properties 
```

### Query Vertices (with parsed row cursor.)
```
ag.queryCypher("MATCH (n) RETURN n")
for vertex in ag.rows():
    print(vertex.id, vertex.properties["name"], vertex) # row has id, label, properties 
```

In [2]:
# Create Vertices
ag.execCypher("CREATE (n:Person {name: 'Joe'})")
ag.execCypher("CREATE (n:Person {name: 'Smith'})")
ag.execCypher("CREATE (n:Person {name: %s})", ('Jack',))
ag.execCypher("CREATE (n:Person {name: 'Andy', title: 'Developer'})")
ag.execCypher("CREATE (n:Person {name: %s, title: %s})", ('Tom','Developer',))
ag.commit()

# Query Vertices with result parsed full graph.
print("-- Query Vertices in full parsed Graph --------------------")
ag.queryCypher("MATCH (n:Person) RETURN n")
graph = ag.graph()
for vertex in graph:
    print(vertex.label, vertex.properties['name'], vertex)

# Query Vertices with parsed row cursor.
print("-- Query Vertices with ag.rows(). --------------------")
ag.queryCypher("MATCH (n:Person) RETURN n")
for vertex in ag.rows():
    print(vertex)
    print(vertex.id, vertex.properties["name"])
    print(vertex.toJson())


-- Query Vertices in full parsed Graph --------------------
Person Joe {label:Person, id:2251799813685384, properties:{name: Joe,}}::VERTEX
Person Smith {label:Person, id:2251799813685385, properties:{name: Smith,}}::VERTEX
Person Jack {label:Person, id:2251799813685386, properties:{name: Jack,}}::VERTEX
Person Andy {label:Person, id:2251799813685387, properties:{name: Andy,title: Developer,}}::VERTEX
Person Tom {label:Person, id:2251799813685388, properties:{name: Tom,title: Developer,}}::VERTEX
-- Query Vertices with ag.rows(). --------------------
{label:Person, id:2251799813685384, properties:{name: Joe,}}::VERTEX
2251799813685384 Joe
{"gtype": "vertex", "label":"Person", "id":2251799813685384, "properties":{"name": "Joe",}}
{label:Person, id:2251799813685385, properties:{name: Smith,}}::VERTEX
2251799813685385 Smith
{"gtype": "vertex", "label":"Person", "id":2251799813685385, "properties":{"name": "Smith",}}
{label:Person, id:2251799813685386, properties:{name: Jack,}}::VERTEX
225

## Create Relation & Query Path

### Create Relations
```
ag.execCypher("MATCH (a:Person), (b:Person) WHERE  a.name = 'Andy' AND b.name = 'Tom' CREATE (a)-[r:workWith]->(b)")
...
ag.commit() # commit
```

### Query Relations (with result parsed full Graph object)
```
ag.queryCypher("MATCH p=()-[:workWith]-() RETURN p") # Cypher Query
graph = ag.graph()
for path in graph:
    print(path)  
```

### Query Relations (with parsed row cursor.)
```
ag.queryCypher("MATCH MATCH p=()-[:workWith]-() RETURN p")
for path in ag.rows():
    print(path)  
```

In [3]:

# Create Edges
ag.execCypher("MATCH (a:Person), (b:Person) WHERE a.name = 'Joe' AND b.name = 'Smith' CREATE (a)-[r:workWith {weight: 3}]->(b)")
ag.execCypher("MATCH (a:Person), (b:Person) WHERE  a.name = 'Andy' AND b.name = 'Tom' CREATE (a)-[r:workWith {weight: 1}]->(b)")
ag.execCypher("MATCH (a:Person {name: 'Jack'}), (b:Person {name: 'Andy'}) CREATE (a)-[r:workWith {weight: 5}]->(b)")
ag.execCypher("MATCH (a:Person {name: 'Joe'}), (b:Person {name: 'Jack'}) CREATE (a)-[r:workWith {weight: 5}]->(b)")
ag.commit()

print("-- [Query path] --------")
ag.queryCypher("MATCH p=()-[:workWith]-() RETURN p")
graph = ag.graph()
for path in graph:
    print(path)
    
print("-- Query graph and print raw string  ----")
from age import rawPrinter
ag.queryCypher("MATCH p=()-[:workWith]-() RETURN p")
graph = ag.graph(resultHandler=rawPrinter)
    

-- [Query path] --------
[{label:Person, id:2251799813685385, properties:{name: Smith,}}::VERTEX,{label:workWith, id:2533274790396544, properties:{weight: 3,}, start_id:2251799813685384, end_id:2251799813685385}::EDGE,{label:Person, id:2251799813685384, properties:{name: Joe,}}::VERTEX]::PATH
[{label:Person, id:2251799813685386, properties:{name: Jack,}}::VERTEX,{label:workWith, id:2533274790396547, properties:{weight: 5,}, start_id:2251799813685384, end_id:2251799813685386}::EDGE,{label:Person, id:2251799813685384, properties:{name: Joe,}}::VERTEX]::PATH
[{label:Person, id:2251799813685384, properties:{name: Joe,}}::VERTEX,{label:workWith, id:2533274790396544, properties:{weight: 3,}, start_id:2251799813685384, end_id:2251799813685385}::EDGE,{label:Person, id:2251799813685385, properties:{name: Smith,}}::VERTEX]::PATH
[{label:Person, id:2251799813685387, properties:{name: Andy,title: Developer,}}::VERTEX,{label:workWith, id:2533274790396546, properties:{weight: 5,}, start_id:225179981

## Query Scalar, properties value

In [4]:
# Query scalar value
print("-- Query scalar value --------------------")
ag.queryCypher("MATCH (n:Person) RETURN id(n)")
for row in ag.rows():
    print(row)
    
# Query properties 
print("-- Query properties --------------------")
ag.queryCypher("MATCH (n:Person) RETURN properties(n)")
for row in ag.rows():
    print(row)
    
# Query properties value
print("-- Query properties value --------------------")
ag.queryCypher("MATCH (n:Person {name: 'Andy'}) RETURN n.title")
for row in ag.rows():
    print(row)
    
print("-- Query path count --------")
ag.queryCypher("MATCH p=(:Person {name: 'Andy'})-[:workWith]-() RETURN count(p)")
graph = ag.graph()
for path in graph:
    print(path)

-- Query scalar value --------------------
2251799813685384
2251799813685385
2251799813685386
2251799813685387
2251799813685388
-- Query properties --------------------
{'name': 'Joe'}
{'name': 'Smith'}
{'name': 'Jack'}
{'name': 'Andy', 'title': 'Developer'}
{'name': 'Tom', 'title': 'Developer'}
-- Query properties value --------------------
Developer
-- Query path count --------
2


## Query SQL

In [5]:
print("-- Query scalar value --------------------")
sql = """
CREATE TABLE IF NOT EXISTS infos (
    user_id serial PRIMARY KEY,
    username VARCHAR ( 50 ) UNIQUE NOT NULL,
    country VARCHAR ( 50 ) NOT NULL
);
"""
ag.execSql(sql)
ag.execSql("INSERT INTO infos(username,country) VALUES ('Joe','US');")
ag.execSql("INSERT INTO infos(username,country) VALUES ('Jack','France');")
ag.execSql("INSERT INTO infos(username,country) VALUES ('Smith','Mexico');")
ag.execSql("INSERT INTO infos(username,country) VALUES ('Andy','Italy');")
ag.commit()

ag.querySql("SELECT * FROM infos")
for row in ag.cursor:
    print(row)

# Not supported 
# ag.querySql("SELECT * FROM infos, cypher(%% MATCH (n:Person) RETURN n %%) as (n agtype) WHERE infos.username=n.name ")
# for row in ag.cursor:
#     print(row)

-- Query scalar value --------------------
(1, 'Joe', 'US')
(2, 'Jack', 'France')
(3, 'Smith', 'Mexico')
(4, 'Andy', 'Italy')


## Close connection

In [6]:
# Clear test data
ag.execCypher("MATCH (n:Person) DETACH DELETE n RETURN *")
ag.execSql("DROP TABLE infos")
ag.commit()

ag.close()